In [1]:
import numpy as np
import quaternion

from intrepid_environment.drone_environment import DroneController

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
# Simulation frequency

DT_MS = 50

In [4]:
controller = DroneController(DT_MS)

Read sensors:

In [5]:
# API Methods

## Read Sensors

# controller.acceleration()
# controller.linear_velocity()
# controller.angular_velocity()
# controller.rotation_angles()

## Control Motors

# controller.control_motors(0.4, 0.7, 0.1, 0.33)

Control drone with a random signal

In [6]:
# for _ in range(0, 5000):
#   thrust = np.random.uniform(0, 200, 4).tolist()
#   try:
#     controller.control_motors(*thrust)
#     await controller.simulation_step()
#   except Exception as e:
#     print("drone crashed")
#     break

# Math Helpers

In [7]:
def skew_symmetric(v):
    """Compute the skew-symmetric matrix of a 3D vector."""
    return np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

def rotation_matrix_i_wrt_b(q):
    """
    Compute the rotation matrix from body frame to inertial frame.
    
    Args:
      q: A numpy auaternion in [w, x, y, z] order.

    Returns:
      A 3x3 numpy array representing the rotation matrix.
    """

    arr = quaternion.as_float_array(q)

    w = arr[0]
    v = arr[1:4]
    Sk = skew_symmetric(v)
    term1 = np.eye(3) * (w**2 - np.dot(v, v))
    term2 = 2 * np.outer(v, v)
    term3 = 2 * w * Sk

    return term1 + term2 + term3

def constrain(value, lower_bound, upper_bound):
    return max(lower_bound, min(upper_bound, value))

## Controllers

In [8]:
# Drone Parameters

## Moment of Inertia

Ixx = 1.0
Iyy = 1.0
Izz = 1.0

mass = 1.0

max_tilt_angle = 1.0
max_ascent_rate = 1.0


In [9]:
# control gain for body_rate_control
kp_pqr = 0.1

def body_rate_control(pqr_cmd, pqr):
    """
    Calculate a desired 3-axis moment given a desired and current body rate

    Args:
      pqr_cmd: desired body rates [rad/s]
      pqr: current or estimated body rates [rad/s]
    Returns:
      A 3x1 numpy array containing the desired moments for each of the 3 axes.
    """

    global Ixx, Iyy, Izz, kp_pqr

    I = np.array([Ixx, Iyy, Izz])
    moment_cmd = I * kp_pqr * (pqr_cmd - pqr)
    return moment_cmd

In [10]:
kp_bank = 1.0


def roll_pitch_yaw_control(accel_cmd, attitude, coll_thrust_cmd):
    """
    Calculate a desired pitch and roll angle rates based on a desired global
            lateral acceleration, the current attitude of the quad, and desired
            collective thrust command

    Args:
        accel_cmd: desired acceleration in global XY coordinates [m/s2]
        attitude: current or estimated attitude of the vehicle
        coll_thrust_cmd: desired collective thrust of the quad [N]
    Returns:
        A 3x1 numpy array containing the desired pitch and roll rates. The Z
                element of the V3F should be left at its default value (0)
    """

    global mass, kp_bank

    pqr_cmd = np.zeros(3)
    R = rotation_matrix_i_wrt_b(attitude)
    if coll_thrust_cmd == 0:
        return pqr_cmd

    coll_accel = -coll_thrust_cmd / mass

    bx_cmd = constrain(accel_cmd[0] / coll_accel, -
                       max_tilt_angle, max_tilt_angle)
    bx_err = bx_cmd - R[0, 2]

    by_cmd = constrain(accel_cmd[1] / coll_accel, -
                       max_tilt_angle, max_tilt_angle)
    by_err = by_cmd - R[1, 2]

    bx_p_term = kp_bank * bx_err
    by_p_term = kp_bank * by_err

    R1 = np.array([
        [
            R[1, 0], -R[0, 0],
        ], [
            R[1, 1], -R[0, 1]
        ]
    ]) / R[2, 2]

    pqr_cmd[0] = R1[0, 0] * bx_p_term + R1[0, 1] * by_p_term
    pqr_cmd[1] = R1[1, 0] * bx_p_term + R1[1, 1] * by_p_term

    return pqr_cmd

In [11]:
integrated_altitude_error = 0.0

kp_pos_z = 1.0
kp_vel_z = 1.0
ki_pos_z = 1.0

max_ascent_rate = 1.0

def altitude_control(pos_z_cmd, vel_z_cmd, pos_z, vel_z, attitude, accel_z_cmd, dt):
    """
	  Calculate desired quad thrust based on altitude setpoint, actual altitude,
	    vertical velocity setpoint, actual vertical velocity, and a vertical 
	    acceleration feed-forward command

    Args:
	    pos_z_cmd, vel_z_cmd: desired vertical position and velocity in NED [m]
	    pos_z, vel_z: current vertical position and velocity in NED [m]
	    accel_z_cmd: feed-forward vertical acceleration in NED [m/s2]
	    dt: the time step of the measurements [seconds]
    Returns:
      A collective thrust command in [N]
    """

    global integrated_altitude_error, kp_pos_z, kp_vel_z, ki_pos_z, max_ascent_rate

    R = rotation_matrix_i_wrt_b(attitude)

    pos_z_err = pos_z_cmd - pos_z
    vel_z_err = vel_z_cmd - vel_z
    integrated_altitude_error += pos_z_err * dt

    b_z = R[2, 2]

    p_term = kp_pos_z * pos_z_err
    d_term = kp_vel_z * vel_z_err
    i_term = ki_pos_z * integrated_altitude_error

    u1_bar = p_term + i_term + d_term + accel_z_cmd

    acc = (u1_bar - 9.81) / b_z
    thrust = -mass * constrain(acc, -max_ascent_rate / dt, max_ascent_rate / dt)

    return thrust

In [12]:
kp_pos_xy = 1.0
kp_vel_xy = 1.0

def lateral_position_control(pos_cmd, vel_cmd, pos, vel, accel_cmd_ff):
    """
	  Calculate a desired horizontal acceleration based on 
	   desired lateral position/velocity/acceleration and current pose

    Args:
	    pos_cmd: desired position, in NED [m]
	    vel_cmd: desired velocity, in NED [m/s]
	    pos: current position, NED [m]
	    vel: current velocity, NED [m/s]
	    accel_cmd_ff: feed-forward acceleration, NED [m/s2]
    Returns:
      A 3x1 numpy array with desired horizontal accelerations. 
        the Z component should be 0
    """

    global kp_pos_xy, kp_vel_xy

    accel_cmd_ff[2] = 0
    vel_cmd[2] = 0
    pos_cmd[2] = pos[2]

    accel_cmd = accel_cmd_ff

    pos_err = pos_cmd - pos
    vel_err = vel_cmd - vel

    accel = kp_pos_xy * pos_err + kp_vel_xy * vel_err + accel_cmd_ff
    accel_cmd[0] = accel[0]
    accel_cmd[1] = accel[1]


    return accel_cmd

In [13]:
kp_yaw = 1.0

def yaw_control(yaw_cmd, yaw):
    """
	  Calculate a desired yaw rate to control yaw to yawCmd

    Args:
	    yaw_cmd: commanded yaw [rad]
	    yaw: current yaw [rad]
    Returns:
      a desired yaw rate [rad/s]
    """

    yaw_rate_cmd = kp_yaw * (yaw_cmd - yaw)

    return yaw_rate_cmd

## Calculating Motor Commands

In [14]:
kappa = 1.0
max_motor_thrust = 100.0
min_motor_thrust = 0.0


def generate_motor_commands(coll_thrust_cmd, moment_cmd):
    """
      Convert a desired 3-axis moment and collective thrust command to 
        individual motor thrust commands
    Args:
      coll_thrust_cmd: desired collective thrust [N]
      moment_cmd: desired rotation moment about each axis [N m]
    Returns:
      a 4-vec with motor commands
    """

    l = np.sqrt(2.0)

    t1 = moment_cmd[0] / l
    t2 = moment_cmd[1] / l
    t3 = -moment_cmd[2] / kappa
    t4 = coll_thrust_cmd

    thrust = np.array([
        ( t1 + t2 + t3 + t4) / 4.0,  # front left  - f1
        (-t1 + t2 - t3 + t4) / 4.0,  # front right - f2
        ( t1 - t2 - t3 + t4) / 4.0,  # rear left   - f4
        (-t1 - t2 + t3 + t4) / 4.0,  # rear right  - f3
    ])

    return thrust

# Test Controllers

## Crazyflie Parameters

In [15]:
# Crazyflie 2.0 physical parameters

mass = 0.03  # kg (typical mass of a Crazyflie 2.0)

length_x = 0.15  # m (dimension along x-axis)
length_y = 0.15  # m (dimension along y-axis)
length_z = 0.05  # m (dimension along z-axis)

# Estimating moments of inertia
Ixx = mass * (length_y**2 + length_z**2) / 12
Iyy = mass * (length_x**2 + length_z**2) / 12
Izz = mass * (length_x**2 + length_y**2) / 12

# Scenario 0: Intro

In [16]:
# set all control gains to zero

kp_pos_z = 0
kp_vel_z = 0
ki_pos_z = 0
kp_pos_xy = 0
kp_vel_xy = 0
kp_yaw = 0
kp_bank = 0
kp_pqr = np.zeros(3)


## Scenario 1: Altitude Control

Tune m parameter until drone stays in place. This scenario using very basic Motor Thrust Controller that equaly distributes thrust to 4 motors.

In [17]:
from intrepid_environment.trajectory import Trajectory, TrajectoryPoint

In [18]:
sim_time = await controller.simulation_time() * 1000
start_time = sim_time
end_time = start_time + 1000

In [19]:
dt = 1

trajectory = Trajectory()

trajectory.add_point(TrajectoryPoint(
    time=sim_time,
    position=np.array([0.0, 0.0, -1.0]),
    velocity=np.zeros(3),
    accel=np.zeros(3),
    attitude=np.quaternion(1.0, 0.0, 0.0, 0.0),
    omega=np.zeros(3)
))

In [20]:
trajectory_time_offset = 0
trajectory_offset = np.zeros(3)

In [21]:
def get_next_trajectory_point(trajectory, sim_time):
    pt = trajectory.next_trajectory_point(sim_time + trajectory_time_offset)
    pt.position += trajectory_offset
    return pt

In [22]:
est_att = np.quaternion(1,0,0,0)
est_omega = np.zeros(3)
est_pos = np.zeros(3)
est_vel = np.zeros(3)


def run_control(dt, sim_time):

    global min_motor_thrust, max_motor_thrust, est_pos, est_vel, est_att, est_omega

    traj_pt = get_next_trajectory_point(trajectory, sim_time)

    (
        cur_trajectory_point_position,
        cur_trajectory_point_velocity,
        cur_trajectory_point_acceleration,
        cur_trajectory_point_attitude,
    ) = traj_pt.position, traj_pt.velocity, traj_pt.accel, traj_pt.attitude

    coll_thrust_cmd = altitude_control(
        cur_trajectory_point_position[2],
        cur_trajectory_point_velocity[2],
        est_pos[2],
        est_vel[2],
        est_att,
        cur_trajectory_point_acceleration[2],
        dt
    )

    # reserve some thrust margin for angle control

    thrust_margin = 0.1 * (max_motor_thrust - min_motor_thrust)
    coll_thrust_cmd = constrain(
        coll_thrust_cmd,
        (min_motor_thrust - thrust_margin) * 4.0,
        (max_motor_thrust + thrust_margin) * 4.0
    )

    des_acc = lateral_position_control(
        cur_trajectory_point_position,
        cur_trajectory_point_velocity,
        est_pos,
        est_vel,
        cur_trajectory_point_acceleration
    )

    cur_trajectory_point_attitude_yaw = quaternion.as_euler_angles(cur_trajectory_point_attitude)[0]
    est_att_yaw = quaternion.as_euler_angles(est_att)[0]
    
    des_omega = roll_pitch_yaw_control(des_acc, est_att, coll_thrust_cmd)
    des_omega[2] = yaw_control(
        cur_trajectory_point_attitude_yaw, est_att_yaw)

    des_moment = body_rate_control(des_omega, est_omega)

    return generate_motor_commands(coll_thrust_cmd, des_moment)

In [23]:
# Drone Parameters

max_ascent_rate = 5

In [ ]:
while sim_time < end_time:
    motor_commands = run_control(dt, sim_time)
    controller.control_motors(*motor_commands)
    await controller.simulation_step()
    sim_time = (await controller.simulation_time()) * 1000
    print(sim_time, end_time)

31250 15626000
46875 15626000
62500 15626000
93750 15626000
109375 15626000
125000 15626000
156250 15626000
171875 15626000
187500 15626000
218750 15626000
234375 15626000
250000 15626000
281250 15626000
296875 15626000
312500 15626000
343750 15626000
359375 15626000
375000 15626000
406250 15626000
421875 15626000
437500 15626000
468750 15626000
484375 15626000
500000 15626000
531250 15626000
546875 15626000
562500 15626000
593750 15626000
609375 15626000
625000 15626000
656250 15626000
671875 15626000
687500 15626000
718750 15626000
734375 15626000
750000 15626000
781250 15626000
796875 15626000
812500 15626000
843750 15626000
859375 15626000
875000 15626000
906250 15626000
921875 15626000
937500 15626000
968750 15626000
984375 15626000
1000000 15626000
1031250 15626000
1046875 15626000
1062500 15626000
1093750 15626000
1109375 15626000
1125000 15626000
1156250 15626000
1171875 15626000
1187500 15626000
1218750 15626000
1234375 15626000
1250000 15626000
1281250 15626000
1296875 156260

ClientDisconnectedError: command 285 canceled due to disconnect